## 필요 라이브러리 호출

In [73]:
import pandas as pd
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
from html_table_parser import parser_functions as parser
import time
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

## 데이터 크롤링

In [6]:
# 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
# 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
# webdriver 설치 위치를 정의한다.
browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')

def crawling(url):
    # 셀레니움(selenium)으로 브라우저를 컨트를하기 위해서는 webdriver를 설치해야 한다
    # 구글크롬은 chromedriver로 검색하면 exe파일을 다운로드 할 수 있다.
    # webdriver 설치 위치를 정의한다.
    browser = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\Application\chromedriver')
    # 불러오고자하는 url을 .get으로 호출한다.
    browser.get(url)
    
    # page_source파라미터를 이용하면 HTML정보를 가지고 온다.
    html = browser.page_source
    soup = BeautifulSoup(html, 'html.parser')   # 뷰티풀숩으로 HTML을 파싱하고 필요한 데이터 수집
    table_tags = soup.find_all("table")   # find_all함수를 이용하면, TABLE태그로 지정된 곳만 뽑아서, 배열 형태로 저장
    table = table_tags[0]    # html상에서 table 순서, 첫번째 테이블 가지고 와야 함.
    p=parser.make2d(table)

    df_total=pd.DataFrame(p[1:],columns=p[0])    # 데이터 프레임으로 저장

    num = soup.find_all("button")[5].text   # 5번째 button에 페이지수 정보가 있음
    for j in range(1, int(num)):
        btn = browser.find_element_by_class_name('paginationWidget-next')
        btn.click()     # 버튼 클릭
        time.sleep(5)

        # 2페이지 이후 데이터 병합
        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')
        table_tags = soup.find_all("table")
        table = table_tags[0]
        p = parser.make2d(table)

        df = pd.DataFrame(p[1:], columns=p[0])
        df_total = pd.concat([df_total, df], 0)
        
    return df_total
        # 엑셀로 다운로드
        # df_total.to_excel("test.xlsx")

In [7]:
list_url = ["http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+hitting&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=hitting&page=1&ts=1580532864437&playerType=ALL",
"http://mlb.mlb.com/stats/sortable.jsp?c_id=mlb#elem=%5Bobject+Object%5D&tab_level=child&click_text=Sortable+Player+fielding&game_type='R'&season=2019&season_type=ANY&league_code='MLB'&sectionType=sp&statType=fielding&page=1&ts=1580539033522&playerType=ALL"]

df_total_hit = crawling(list_url[0])
df_total_field = crawling(list_url[1])

## 전처리 진행
- 불필요한 컬럼 삭제(G, GS) , RK, Team, 빈 컬럼, 등
- hit & field data mergy : key column Player & Team $ Position
- 1B, 2B, 3B, SS 포지션만 추출
- AB 타석수(20), INN(20)가 일정 값 이상의 선수만 대상으로 한다.


In [8]:
# 컬럼 rename
df_total_hit = df_total_hit.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', 
                                            '':'del', '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'AB▼':'AB', 'R▼':'R',
                                            'H▼':'H', '2B▼':'2B', '3B▼':'3B', 'HR▼':'HR',
                                            'RBI▼':'RBI', 'BB▼':'BB', 'SO▼':'SO', 'SB▼':'SB',
                                            'CS▼':'CS', 'AVG▼':'AVG', 'OBP▼':'OBP',
                                            'SLG▼':'SLG', 'OPS▼':'OPS', 'IBB▼':'IBB',
                                            'HBP▼':'HBP', 'SAC▼':'SAC', 'SF▼':'SF', 
                                            'TB▼':'TB', 'XBH▼':'XBH', 'GDP▼':'GDP',
                                            'GO▼':'GO', 'AO▼':'AO', 'GO_AO▼':'GO_AO',
                                            'NP▼':'NP', 'PA▼':'PA'})
df_total_field = df_total_field.rename(columns={'RK':'RK', 'Player':'Player', 'Team':'Team', '':'del',
                                            '▲':'del2', 'Pos':'Pos', 'G▼':'G', 'GS▼':'GS',
                                            'INN▼':'INN', 'TC▼':'TC', 'PO▼':'PO', 'A▼':'A',
                                            'E▼':'E', 'DP▼':'DP', 'SB▼':'SB', 'CS▼':'CS',
                                            'SBPCT▲':'SBPCT', 'PB▼':'PB', 'C_WP▼':'C_WP',
                                            'FPCT▼':'FPCT', 'RF▼':'RF'})

In [9]:
df_total_hit.to_excel("df_total_hit.xlsx")
df_total_hit

,RK,Player,Team,del,del2,Pos,G,AB,R,H,...,SAC,SF,TB,XBH,GDP,GO,AO,GO_AO,NP,PA
0,1,"Alaniz, R",CIN,,595798,P,8,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
1,1,"Brebbia, J",STL,,605154,P,63,1,1,1,...,0,0,1,0,0,0,0,-.--,1,1
2,1,"Drake, O",TB,,543118,P,3,1,0,1,...,0,0,1,0,0,0,0,-.--,4,1
3,1,"Liriano, F",PIT,,434538,P,67,1,0,1,...,0,0,1,0,0,0,0,-.--,15,2
4,1,"Lugo, S",NYM,,607625,P,57,1,0,1,...,0,0,1,0,0,0,0,-.--,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32,984,"Yates, K",SD,,489446,P,59,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
33,984,"Ynoa, G",BAL,,593679,P,4,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
34,984,"Ynoa, H",ATL,,660623,P,2,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0
35,984,"Zamora, D",NYM,,623354,P,17,0,0,0,...,0,0,0,0,0,0,0,-.--,0,0


In [10]:
df_total_field.to_excel("df_total_field.xlsx")
df_total_field

,RK,Player,Team,del,del2,Pos,G,GS,INN,TC,...,A,E,DP,SB,CS,SBPCT,PB,C_WP,FPCT,RF
0,1,"Abreu, B",HOU,,650556,P,7,0,8.2,4,...,2,0,0,-,-,-,-,-,1.000,0.57
1,1,"Adames, C",SF,,542436,SS,7,5,46.0,15,...,8,0,2,-,-,-,-,-,1.000,2.14
2,1,"Adams, A",DET,,542866,P,15,0,16.2,4,...,3,0,0,-,-,-,-,-,1.000,0.27
3,1,"Adams, C",NYY,,664856,P,13,0,25.1,2,...,2,0,0,-,-,-,-,-,1.000,0.15
4,1,"Adduci, J",CHC,,451192,RF,1,1,8.0,5,...,0,0,0,-,-,-,-,-,1.000,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1319,"Walker, T",ARI,,592836,P,1,1,1.0,0,...,0,0,0,-,-,-,-,-,-,0.00
1,1319,"Warren, A",SEA,,605521,P,6,0,5.1,0,...,0,0,0,-,-,-,-,-,-,0.00
2,1319,"Williams, A",WSH,,657117,P,2,0,0.1,0,...,0,0,0,-,-,-,-,-,-,0.00
3,1319,"Williams, D",MIL,,642207,P,13,0,13.2,0,...,0,0,0,-,-,-,-,-,-,0.00


In [14]:
df_total_hit_ = pd.read_excel("df_total_hit.xlsx")
df_total_field_ = pd.read_excel("df_total_field.xlsx")

In [33]:
# 데이터 백업

player_hit_data = df_total_hit_
player_field_data = df_total_field_
player_hit_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1287 entries, 0 to 1286
Data columns (total 35 columns):
Unnamed: 0    1287 non-null int64
RK            1287 non-null int64
Player        1287 non-null object
Team          1287 non-null object
del           0 non-null float64
del2          1287 non-null int64
Pos           1287 non-null object
G             1287 non-null int64
AB            1287 non-null int64
R             1287 non-null int64
H             1287 non-null int64
2B            1287 non-null int64
3B            1287 non-null int64
HR            1287 non-null int64
RBI           1287 non-null int64
BB            1287 non-null int64
SO            1287 non-null int64
SB            1287 non-null int64
CS            1287 non-null int64
AVG           1287 non-null object
OBP           1287 non-null object
SLG           1287 non-null object
OPS           1287 non-null float64
IBB           1287 non-null int64
HBP           1287 non-null int64
SAC           1287 non-null int64
SF

In [34]:
# 불필요한 컬럼 삭제

player_hit_data = player_hit_data.drop(['Unnamed: 0', 'RK', 'del', 'del2', 'GO_AO', 'G'], axis=1)
player_field_data = player_field_data.drop(['Unnamed: 0', 'RK', 'del', 'del2', 'G', 'GS', 'SB', 'CS', 'SBPCT','PB','C_WP'], axis=1)

In [35]:
# column Player & Team $ Position 기준 병합

df_total = pd.merge(player_hit_data, player_field_data, on=['Player', 'Team', 'Pos']) # how='outer')
df_total.columns
#df_total.to_excel("test.xlsx")

Index(['Player', 'Team', 'Pos', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'BB',
       'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'IBB', 'HBP', 'SAC', 'SF',
       'TB', 'XBH', 'GDP', 'GO', 'AO', 'NP', 'PA', 'INN', 'TC', 'PO', 'A', 'E',
       'DP', 'FPCT', 'RF'],
      dtype='object')

In [36]:
# 컬럼 조건문으로 행 추출하기 Position 컬럼에서 "1B", "2B", "3B", "SS"

# 하나의 조건문이라면 "해당 컬럼 == 하나조건"으로 하면 되지만, "해당컬럼.isin([여러조건])"
# df_total_hit_infld = df_total_hit[df_total_hit['Pos'] == "2B"]
df_total_infld = df_total[df_total['Pos'].isin(["1B", "2B", "3B", "SS"])]
df_total_infld
print(df_total_infld.shape)

(288, 37)


In [37]:
# df_total_infld.info()
# df_total_infld.dtypes 데이터 타입만 보려면

### 이렇게 해서 데이터 타입을 바꾸니 문제가 있네
```
# object --> numeric(int or float) 
# 현재 데이터는 전부 object type이여서 수치형 데이터는 pd.to_numeric함수를 이용해 변환
# 데이터프레임에서 컬럼으로 슬라이싱하려면 iloc을 사용
# df_total_infld['G_x'] = pd.to_numeric(df_total_infld['G_x'])
df_total_infld = df_total_infld.apply(pd.to_numeric.iloc, errors='coerce')

# 문자열 타입인데 numeric 타입으로 바뀐것을 원상 복구
# numeric(int or float) --> object
df_total_infld.astype({'Player':'str', 'Team':'str', 'Pos':'str'}).info()
```
- object가 numeric으로 바뀌니깐 NaN값이 되어 버림

- list로 해서 데이터 타입 일괄로 numeric으로 바꾸기
```
col_list = ['AB', 'R', 'H', '2B', '3B', 'HR', 'RBI',
       'BB', 'SO', 'SB', 'CS', 'AVG', 'OBP', 'SLG', 'OPS', 'IBB', 'HBP', 'SAC',
       'SF', 'TB', 'XBH', 'GDP', 'GO', 'AO', 'NP', 'PA', 'INN',
       'TC', 'PO', 'A', 'E', 'DP', 'FPCT', 'RF']
for i in col_list:
    df_total_infld[i] = pd.to_numeric(df_total_infld[i], errors='coerce')
```

In [38]:
# 타석수와 수비이닝의 최소값 제한

df_total_infld = df_total_infld[df_total_infld['AB'] >= 20]
print(df_total_infld.shape)
df_total_infld = df_total_infld[df_total_infld['INN'] >= 20]
print(df_total_infld.shape)

(274, 37)
(273, 37)


In [45]:
# Null 여부 확인
df_total_infld.isnull().count()

Player    273
Team      273
Pos       273
AB        273
R         273
H         273
2B        273
3B        273
HR        273
RBI       273
BB        273
SO        273
SB        273
CS        273
AVG       273
OBP       273
SLG       273
OPS       273
IBB       273
HBP       273
SAC       273
SF        273
TB        273
XBH       273
GDP       273
GO        273
AO        273
NP        273
PA        273
INN       273
TC        273
PO        273
A         273
E         273
DP        273
FPCT      273
RF        273
dtype: int64

- 특정 컬럼의 null값이 몇개 있는지 확인
```
df_total_infld.iloc[:,3:]
print(df_total_infld['FPCT'].isnull().value_counts())
```

- NaN값을 N으로 변경
```
df_total_infld['FPCT'].fillna('N', inplace=True)
df_total_infld.info()
```

## 데이터 분석하기
- groupby를 이용하여 분석하기


In [46]:
df_total_infld

,Player,Team,Pos,AB,R,H,2B,3B,HR,RBI,...,NP,PA,INN,TC,PO,A,E,DP,FPCT,RF
22,"Kendrick, H",WSH,3B,334,61,115,23,1,17,62,...,1397,370,601.0,401,308,91,2,35,.995,4.29
23,"Anderson, T",CWS,SS,498,81,167,32,0,18,56,...,1765,518,1050.0,532,182,324,26,75,.951,4.11
24,"Arraez, L",MIN,2B,326,54,109,20,1,4,28,...,1486,366,716.0,227,101,121,5,31,.978,2.22
37,"Solano, D",SF,SS,215,27,71,13,1,4,23,...,903,228,407.0,199,84,113,2,27,.990,3.46
40,"LeMahieu, D",NYY,2B,602,109,197,33,2,26,102,...,2463,655,1241.2,620,351,261,8,63,.987,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,"Saladino, T",MIL,SS,65,7,8,0,0,2,8,...,287,71,151.0,62,28,33,1,7,.984,2.10
688,"Barreto, F",OAK,2B,57,6,7,2,0,2,5,...,237,58,146.2,71,20,46,5,8,.930,3.00
711,"Pedroia, D",BOS,2B,20,1,2,0,0,0,1,...,77,21,28.0,16,9,7,0,3,1.000,2.67
727,"Velazquez, A",CLE,3B,23,3,2,2,0,0,0,...,111,24,55.0,28,13,14,1,3,.964,2.25


In [60]:
# Playe와 Team 컬럼 삭제

df_total_infld.drop(['Player', 'Team'], axis=1, inplace=True)

In [61]:
# 포지션 별로 각 수치 평균

df_total_infld.groupby('Pos').agg(['mean'])

,AB,R,H,2B,3B,HR,RBI,BB,SO,SB,...,AO,NP,PA,INN,TC,PO,A,E,DP,RF
,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean,...,mean,mean,mean,mean,mean,mean,mean,mean,mean,mean
Pos,,,,,,,,,,,,,,,,,,,,,
1B,304.553846,43.692308,75.846154,15.892308,0.800000,15.000000,47.707692,37.061538,81.338462,1.323077,...,77.738462,1415.000000,347.969231,602.364615,532.092308,479.738462,48.061538,4.292308,47.646154,5.406923
2B,272.723684,37.118421,70.500000,13.434211,1.552632,8.947368,33.078947,23.144737,60.171053,4.500000,...,72.052632,1164.934211,301.776316,603.382895,271.144737,117.710526,147.868421,5.565789,35.802632,2.968684
3B,295.513514,42.810811,77.324324,16.256757,1.378378,13.013514,42.959459,29.445946,71.486486,2.716216,...,78.337838,1307.081081,331.243243,642.674324,225.743243,95.445946,123.500000,6.797297,18.081081,2.554054
SS,321.327586,46.034483,85.879310,17.758621,1.982759,11.741379,41.103448,24.982759,71.362069,6.706897,...,82.672414,1348.086207,351.810345,712.108621,311.517241,103.655172,199.810345,8.051724,42.155172,3.162414


In [66]:
df_total_infld

,Pos,AB,R,H,2B,3B,HR,RBI,BB,SO,...,NP,PA,INN,TC,PO,A,E,DP,FPCT,RF
22,3B,334,61,115,23,1,17,62,27,49,...,1397,370,601.0,401,308,91,2,35,.995,4.29
23,SS,498,81,167,32,0,18,56,15,109,...,1765,518,1050.0,532,182,324,26,75,.951,4.11
24,2B,326,54,109,20,1,4,28,36,29,...,1486,366,716.0,227,101,121,5,31,.978,2.22
37,SS,215,27,71,13,1,4,23,10,49,...,903,228,407.0,199,84,113,2,27,.990,3.46
40,2B,602,109,197,33,2,26,102,46,90,...,2463,655,1241.2,620,351,261,8,63,.987,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,SS,65,7,8,0,0,2,8,5,26,...,287,71,151.0,62,28,33,1,7,.984,2.10
688,2B,57,6,7,2,0,2,5,1,23,...,237,58,146.2,71,20,46,5,8,.930,3.00
711,2B,20,1,2,0,0,0,1,1,2,...,77,21,28.0,16,9,7,0,3,1.000,2.67
727,3B,23,3,2,2,0,0,0,1,13,...,111,24,55.0,28,13,14,1,3,.964,2.25


In [67]:
# label 데이터 레이블 인코딩
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(df_total_infld['Pos'])
df_total_infld['Pos'] = encoder.transform(df_total_infld['Pos'])
#labels = labels.reshape(-1, 1)
df_total_infld

,Pos,AB,R,H,2B,3B,HR,RBI,BB,SO,...,NP,PA,INN,TC,PO,A,E,DP,FPCT,RF
22,2,334,61,115,23,1,17,62,27,49,...,1397,370,601.0,401,308,91,2,35,.995,4.29
23,3,498,81,167,32,0,18,56,15,109,...,1765,518,1050.0,532,182,324,26,75,.951,4.11
24,1,326,54,109,20,1,4,28,36,29,...,1486,366,716.0,227,101,121,5,31,.978,2.22
37,3,215,27,71,13,1,4,23,10,49,...,903,228,407.0,199,84,113,2,27,.990,3.46
40,1,602,109,197,33,2,26,102,46,90,...,2463,655,1241.2,620,351,261,8,63,.987,3.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687,3,65,7,8,0,0,2,8,5,26,...,287,71,151.0,62,28,33,1,7,.984,2.10
688,1,57,6,7,2,0,2,5,1,23,...,237,58,146.2,71,20,46,5,8,.930,3.00
711,1,20,1,2,0,0,0,1,1,2,...,77,21,28.0,16,9,7,0,3,1.000,2.67
727,2,23,3,2,2,0,0,0,1,13,...,111,24,55.0,28,13,14,1,3,.964,2.25


In [68]:
# 데이터 세트 분리 (피처 데이터 and 라벨 데이터)

y_infield_df = df_total_infld['Pos']
x_infield_df = df_total_infld.drop('Pos', axis=1)

In [72]:
# train_test_split()를 통해 train과 test 데이터 분리
# from sklearn.model_selection import train_test_split

x_train, y_train, x_test, y_test = train_test_split(x_infield_df, y_infield_df, test_size=0.2, random_state=11)

In [77]:

x_train.to_excel("x_train.xlsx")

## 테스트로 2가지 모델로 학습하고 예측/평가해보자
- Random Forest
- Decision Tree

In [74]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import accuracy_score
deci_tree = DecisionTreeClassifier(random_state=11)
rand_fore = RandomForestClassifier(random_state=11)

# Decision Tree
deci_tree.fit(x_train, y_train)
deci_tree_pred = deci_tree.predict(x_test)
print('결정트리 정확도: {0:.4f}'.format(accuracy_score(y_test, deci_tree_pred)))

# Random Forest
rand_fore.fit(x_train, y_train)
rand_fore_pred = rand_fore.predict(x_test)
print('랜덤포레스트의 정확도:{1:.4f}'.format(accuracy_score(y_test, rand_fore_pred)))

TypeError: '<' not supported between instances of 'float' and 'str'